In [1]:
## import libraries
import os, sys
import yaml
import re
import xarray as xr
import pandas as pd
import numpy as np
from datetime import timedelta
%matplotlib inline
from datetime import timedelta

# plotting
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.gridspec import GridSpec
from matplotlib.colorbar import Colorbar # different way to handle colorbar
import textwrap
import cmocean.cm as cmo

# import personal modules
sys.path.append('../modules')
import custom_cmaps as ccmap
from plotter import draw_basemap
import ar_funcs
import mclimate_funcs as mclim_func
# dask.config.set(**{'array.slicing.split_large_chunks': True})

ERROR 1: PROJ: proj_create_from_database: Open of /home/dnash/miniconda3/envs/SEAK-impacts/share/proj failed


In [2]:
path_to_data = '/expanse/nfs/cw3e/cwp140/'      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
## get list of impact dates broken down by subregion
df = ar_funcs.clean_impact_data(start_date = '2000-01-01', end_date = '2019-08-31')

## Group together
group_name = 'Northeast Gulf Coast'

if group_name == 'Northeast Gulf Coast':
    zone = "northern_coastal"
    idx = (df['Location'] == 'PAYA')
elif group_name == 'Central East Gulf Coast':
    zone = "central_coastal"
    idx = (df['Location'] == 'PASI')
elif group_name == 'Southeast Gulf Coast':
    zone = "southern_coastal"
    idx = (df['Location'] == 'PAKW')
elif group_name == 'Northern Inner Channels':
    zone = "northern_inner_channel"
    idx = (df['Location'] == 'PAHN') | (df['Location'] == 'COOPHCSA2') | (df['Location'] == 'PAGY')
elif group_name == 'Central Inner Channels':
    zone = "central_inner_channel"
    idx = (df['Location'] == 'PAGS') | (df['Location'] == 'PAJN') | (df['Location'] == 'HONA2') | (df['Location'] == 'PAPG') | (df['Location'] == 'PAWG')
    # 46 23 1
elif group_name == 'Southern Inner Channels':
    zone = "southern_inner_channel"
    idx = (df['Location'] == 'PAKT') | (df['Location'] == 'KTNA2')

df = df.loc[idx]

idx = (df['Impact Level'] >=4)
dates_high = pd.to_datetime(df.loc[idx].index.values).unique()

idx = (df['Impact Level'] >1) & (df['Impact Level'] <4)
dates_medium = pd.to_datetime(df.loc[idx].index.values).unique()

idx = (df['Impact Level'] <=1)
dates_low = pd.to_datetime(df.loc[idx].index.values).unique()

dates_lst = [dates_low, dates_medium, dates_high]
print(len(dates_low), len(dates_medium), len(dates_high))

16 0 0


In [4]:
# ## load the preprocessed mclimate comparison to AR dates at 72 hour lead
# fname_pattern = path_to_data + 'preprocessed/mclimate_AR_dates/mclimate_ivt_*_F72.nc'
# ds = xr.open_mfdataset(fname_pattern, engine='netcdf4', combine='nested', concat_dim="valid_time")

# data_lst = []
# for i, dt_vals in enumerate(dates_lst):
#     tmp = ds.sel(valid_time=dt_vals) # subset to high/medium/low impact dates
#     # tmp = tmp.isel(valid_time=100)
#     tmp = tmp.mean('valid_time') # average the composite
#     data_lst.append(tmp)

In [5]:
data_lst = []
for i, dt_vals in enumerate(dates_lst):
    ## get string values for opening datasets
    dates_new = []
    mon_lst = []
    day_lst = []
    for i, date in enumerate(dt_vals):
        # ts = pd.to_datetime(str(date))
        ts = date
        d = ts - timedelta(days=3)
        t = d.strftime('%Y%m%d')
        mon_lst.append(d.month)
        day_lst.append(d.day)
        dates_new.append(t)

    ## now compare mclimate to each forecast
    ds_lst = []
    for j, dates in enumerate(dates_new):
        try:
            fc = mclim_func.load_reforecast(dates, 'ivt')
            mclimate = mclim_func.load_mclimate(mon_lst[j], day_lst[j])
            
            ds = mclim_func.compare_mclimate_to_forecast(fc, mclimate)
            ds_lst.append(ds)
        except OSError:
            pass
    
    ds_final = xr.concat(ds_lst, dim="impact_dates")
    data = ds_final.mean('impact_dates')
    data_lst.append(data)

ValueError: must supply at least one object to concatenate

In [ ]:
np.nanmin(data_lst[1].ivt_mclimate.values)

In [ ]:
data_lst[1]

In [ ]:
# Set up projection
mapcrs = ccrs.Mercator()
# mapcrs = ccrs.PlateCarree()
datacrs = ccrs.PlateCarree()

# Set tick/grid locations
lats = data_lst[0].lat.values
lons = data_lst[0].lon.values
dx = np.arange(lons.min().round(),lons.max().round()+10,10)
dy = np.arange(lats.min().round(),lats.max().round()+5,5)

ext = [-170., -120., 40., 65.]

In [ ]:
# Create figure
fig = plt.figure(figsize=(19, 5))
fig.dpi = 300
fname = path_to_figs + 'impact_NWS_composite_{0}'.format(zone)
fmt = 'png'

nrows = 1
ncols = 4

# contour labels
kw_clabels = {'fontsize': 7, 'inline': True, 'inline_spacing': 7, 'fmt': '%i',
              'rightside_up': True, 'use_clabeltext': True}

kw_ticklabels = {'size': 10, 'color': 'dimgray', 'weight': 'light'}

## Use gridspec to set up a plot with a series of subplots that is
## n-rows by n-columns
gs = GridSpec(nrows, ncols, height_ratios=[1], width_ratios = [1, 1, 1, 0.05], wspace=0.001, hspace=0.05)
## use gs[rows index, columns index] to access grids

###################
### CLIMATOLOGY ###
###################
leftlats_lst = [True, False, False]
lbl = ['Low Impact', 'Medium Impact', 'High Impact']
for i, fc in enumerate(data_lst):
    ax = fig.add_subplot(gs[0, i], projection=mapcrs) 
    ax = draw_basemap(ax, extent=ext, xticks=dx, yticks=dy, left_lats=leftlats_lst[i], right_lats=False, bottom_lons=True)
    
    # Contour Filled
    data = fc.ivt_mclimate.values*100.
    # cmap, norm, bnds = ccmap.cmap('mclimate_purple')
    # cf = ax.contourf(fc.lon, fc.lat, data, transform=datacrs,
    #                  levels=bnds, cmap=cmap, norm=norm, alpha=0.9, extend='neither')

    cmap = cmo.deep
    bnds = np.arange(75, 100.5, 0.5)
    cf = ax.contourf(fc.lon.values, fc.lat.values, data, transform=datacrs,
                     levels=bnds, cmap=cmap, alpha=0.9, extend='both')
    ax.set_title(lbl[i], loc='left', fontsize=10)
    
    # # Contour Lines
    # clevs = np.arange(250., 2100., 250.)
    # cs = ax.contour(ds.lon, ds.lat, ds.IVT, transform=datacrs,
    #                  levels=clevs, colors='k',
    #                  linewidths=0.75, linestyles='solid')
    # plt.clabel(cs, **kw_clabels)
    
    # ts = pd.to_datetime(str(ds.time.values)) 
    # init_time = ts.strftime('%HZ %d %b %Y')
    # start_date = ts - timedelta(days=45)
    # start_date = start_date.strftime('%d-%b')
    # end_date = ts + timedelta(days=45)
    # end_date = end_date.strftime('%d-%b')
    
    # ts = pd.to_datetime(str(ds.valid_time.values)) 
    # valid_time = ts.strftime('%HZ %d %b %Y')
    
    # ax.set_title('Model Run: {0}'.format(init_time), loc='left', fontsize=10)
    # ax.set_title('Valid Date: {0}'.format(valid_time), loc='right', fontsize=10)

    
    # txt = 'Relative to all 162-h GEFSv12 reforecasts initialized between {0} and {1} (2000-2019)'.format(start_date, end_date)
    # ann_ax = fig.add_subplot(gs[-1, i])
    # ann_ax.axis('off')
    # ann_ax.annotate(textwrap.fill(txt, 60), # this is the text
    #            (0, 0.), # these are the coordinates to position the label
    #             textcoords="offset points", # how to position the text
    #             xytext=(25,-35), # distance from text to points (x,y)
    #             ha='left', # horizontal alignment can be left, right or center
    #             **kw_ticklabels)


# Add color bar
cbax = plt.subplot(gs[0,-1]) # colorbar axis
cb = Colorbar(ax = cbax, mappable = cf, orientation = 'vertical', ticklocation = 'right')
cb.set_label('Model Climate Percentile Rank (xth)', fontsize=10)
cb.ax.tick_params(labelsize=8)

fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi)

# Show
plt.show()